In [1]:
import pandas as pd
import numpy as  np

from TargetEncoderv2 import TargetEncoder
from FeatureSelector import FeatureSelector

from sklearn.metrics import *
from sklearn.model_selection import *

import lightgbm as lgb

In [2]:
dtypes = dtype = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

train = pd.read_csv("../input/train.csv", skiprows = list(range(1,140000000)))

/home/mohsin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Get hour information
train["click_time"] = pd.to_datetime(train["click_time"])
train["hourofday"] = train["click_time"].dt.hour
train["minuteofhour"] = train["click_time"].dt.minute

In [4]:
from utils import outoffold_crossvalidator, shuffle_crossvalidator
cvlist1 = list(StratifiedShuffleSplit(1, test_size=0.05, random_state=3).split(train, train.is_attributed))


In [6]:
model = lgb.LGBMClassifier(num_leaves=31, n_jobs=-1, n_estimators=500, subsample=0.9, colsample_bytree=0.95,
                           verbose=10)

In [7]:
%%time
features = ['ip', 'app', 'device', 'os', 'channel', 'hourofday']
shuffle_crossvalidator(model, train[features], train.is_attributed, cvlist=cvlist1)

Training until validation scores don't improve for 50 rounds.
[10]	valid_0's auc: 0.910545
[20]	valid_0's auc: 0.913077
[30]	valid_0's auc: 0.926083
[40]	valid_0's auc: 0.946944
[50]	valid_0's auc: 0.954194
[60]	valid_0's auc: 0.959343
[70]	valid_0's auc: 0.960749
[80]	valid_0's auc: 0.961829
[90]	valid_0's auc: 0.962673
[100]	valid_0's auc: 0.963272
[110]	valid_0's auc: 0.963692
[120]	valid_0's auc: 0.963465
[130]	valid_0's auc: 0.963536
[140]	valid_0's auc: 0.963905
Early stopping, best iteration is:
[98]	valid_0's auc: 0.964441
Score for this fold is  0.9644412205344001
Overall score on 10 fold CV is 0.9644412205344001
CPU times: user 26min 32s, sys: 5.92 s, total: 26min 38s
Wall time: 3min 8s


In [8]:
%%time
cnt_enc = TargetEncoder(cols=['device'], targetcol='is_attributed', func='count', cname='device_cnt', add_to_orig=True)
train = cnt_enc.fit_transform(train)

device_cnt
CPU times: user 4.72 s, sys: 3.9 s, total: 8.62 s
Wall time: 8.64 s


In [9]:
%%time
cnt_enc = TargetEncoder(cols=['os'], targetcol='is_attributed', func='count', cname='os_cnt', add_to_orig=True)
train = cnt_enc.fit_transform(train)

os_cnt
CPU times: user 6.09 s, sys: 7.23 s, total: 13.3 s
Wall time: 20.8 s


In [10]:
%%time
cnt_enc = TargetEncoder(cols=['ip'], targetcol='is_attributed', func='count', cname='ip_cnt', add_to_orig=True)
train = cnt_enc.fit_transform(train)

ip_cnt
CPU times: user 10.2 s, sys: 7.06 s, total: 17.2 s
Wall time: 21.6 s


In [11]:
%%time
cnt_enc = TargetEncoder(cols=['app'], targetcol='is_attributed', func='count', cname='app_cnt', add_to_orig=True)
train = cnt_enc.fit_transform(train)

app_cnt
CPU times: user 6.86 s, sys: 7.74 s, total: 14.6 s
Wall time: 20.6 s


In [12]:
%%time
cnt_enc = TargetEncoder(cols=['channel'], targetcol='is_attributed', func='count', cname='channel_cnt', add_to_orig=True)
train = cnt_enc.fit_transform(train)

channel_cnt
CPU times: user 7.09 s, sys: 8.23 s, total: 15.3 s
Wall time: 24.9 s


In [13]:
import gc
gc.collect()

148

In [14]:
%%time
features = ['ip', 'app', 'device', 'os', 'channel', 'device_cnt', 'ip_cnt', 'os_cnt', 'ip_cnt', 'app_cnt', 'channel_cnt', 
            'hourofday', 'minuteofhour']
shuffle_crossvalidator(model, train[features], train.is_attributed, cvlist=cvlist1)

Training until validation scores don't improve for 50 rounds.
[10]	valid_0's auc: 0.922105
[20]	valid_0's auc: 0.922446
[30]	valid_0's auc: 0.925776
[40]	valid_0's auc: 0.929605
[50]	valid_0's auc: 0.960451
[60]	valid_0's auc: 0.960445
[70]	valid_0's auc: 0.96596
[80]	valid_0's auc: 0.967531
[90]	valid_0's auc: 0.969267
[100]	valid_0's auc: 0.970039
[110]	valid_0's auc: 0.970699
[120]	valid_0's auc: 0.970982
[130]	valid_0's auc: 0.971173
[140]	valid_0's auc: 0.971354
[150]	valid_0's auc: 0.971487
[160]	valid_0's auc: 0.971571
[170]	valid_0's auc: 0.971475
[180]	valid_0's auc: 0.971424
[190]	valid_0's auc: 0.971517
[200]	valid_0's auc: 0.971516
[210]	valid_0's auc: 0.97146
Early stopping, best iteration is:
[168]	valid_0's auc: 0.971631
Score for this fold is  0.9716309520812434
Overall score on 10 fold CV is 0.9716309520812434
CPU times: user 47min 45s, sys: 11.3 s, total: 47min 56s
Wall time: 5min 23s


In [15]:
#train["hour_sine"] = np.sin((train["hourofday"] + train['minuteofhour']/60.0)/24 * 2* np.pi)
#train["hour_cosine"] = np.cos((train["hourofday"] + train['minuteofhour']/60.0)/24 * 2* np.pi)

In [16]:
%%time
#features = ['ip', 'app', 'device', 'os', 'channel', 'device_cnt', 'ip_cnt', 'os_cnt', 'ip_cnt', 'app_cnt', 'channel_cnt', 
#            'hourofday', 'minuteofhour', 'hour_sine', 'hour_cosine']
#shuffle_crossvalidator(model, train[features], train.is_attributed, cvlist=cvlist1)

Training until validation scores don't improve for 50 rounds.
[10]	valid_0's auc: 0.92222
[20]	valid_0's auc: 0.930365
[30]	valid_0's auc: 0.929721
[40]	valid_0's auc: 0.931441
[50]	valid_0's auc: 0.961258
[60]	valid_0's auc: 0.96316
[70]	valid_0's auc: 0.96652
[80]	valid_0's auc: 0.968365
[90]	valid_0's auc: 0.969502
[100]	valid_0's auc: 0.970227
[110]	valid_0's auc: 0.97026
[120]	valid_0's auc: 0.970537
[130]	valid_0's auc: 0.970564
[140]	valid_0's auc: 0.970645
[150]	valid_0's auc: 0.970368
[160]	valid_0's auc: 0.970477
[170]	valid_0's auc: 0.970615
[180]	valid_0's auc: 0.970697
[190]	valid_0's auc: 0.970704
[200]	valid_0's auc: 0.970696
[210]	valid_0's auc: 0.970688
[220]	valid_0's auc: 0.970681
[230]	valid_0's auc: 0.970614
Early stopping, best iteration is:
[188]	valid_0's auc: 0.970718
Score for this fold is  0.9707183941251114
Overall score on 10 fold CV is 0.9707183941251114
CPU times: user 49min 38s, sys: 13.4 s, total: 49min 52s
Wall time: 5min 56s


In [15]:
%%time
cnt_enc = TargetEncoder(cols=['device', 'hourofday'], targetcol='is_attributed', func='count', cname='device_hour_cnt', add_to_orig=True)
train = cnt_enc.fit_transform(train)
cnt_enc = TargetEncoder(cols=['ip', 'hourofday'], targetcol='is_attributed', func='count', cname='ip_hour_cnt', add_to_orig=True)
train = cnt_enc.fit_transform(train)
cnt_enc = TargetEncoder(cols=['channel', 'hourofday'], targetcol='is_attributed', func='count', cname='channel_hour_cnt', add_to_orig=True)
train = cnt_enc.fit_transform(train)
cnt_enc = TargetEncoder(cols=['os', 'hourofday'], targetcol='is_attributed', func='count', cname='os_hour_cnt', add_to_orig=True)
train = cnt_enc.fit_transform(train)
cnt_enc = TargetEncoder(cols=['app','hourofday'], targetcol='is_attributed', func='count', cname='app_hour_cnt', add_to_orig=True)
train = cnt_enc.fit_transform(train)

device_hour_cnt
ip_hour_cnt
channel_hour_cnt
os_hour_cnt
app_hour_cnt
CPU times: user 50.4 s, sys: 1min 1s, total: 1min 52s
Wall time: 3min 27s


In [ ]:
%%time
features = ['ip', 'app', 'device', 'os', 'channel', 'device_cnt', 'ip_cnt', 'os_cnt', 'ip_cnt', 'app_cnt', 'channel_cnt', 
            'hourofday', 'minuteofhour', 'device_hour_cnt','ip_hour_cnt', 'channel_hour_cnt','os_hour_cnt','app_hour_cnt' ]
shuffle_crossvalidator(model, train[features], train.is_attributed, cvlist=cvlist1)

Training until validation scores don't improve for 50 rounds.
[10]	valid_0's auc: 0.922865
[20]	valid_0's auc: 0.92633
[30]	valid_0's auc: 0.926615
[40]	valid_0's auc: 0.930931
[50]	valid_0's auc: 0.95844


In [ ]:
cnt_enc = TargetEncoder(cols=['hourofday'], targetcol='is_attributed', func='count', cname='hour_cnt', add_to_orig=True)
train = cnt_enc.fit_transform(train)

In [ ]:
train["device_hour_cntphour"] = train["device_hour_cnt"]/train["hour_cnt"]
train["os_hour_cntphour"] = train["os_hour_cnt"]/train["hour_cnt"]
train["app_hour_cntphour"] = train["app_hour_cnt"]/train["hour_cnt"]
train["ip_hour_cntphour"] = train["ip_hour_cnt"]/train["hour_cnt"]
train["channel_hour_cntphour"] = train["channel_hour_cnt"]/train["hour_cnt"]

In [ ]:
import gc
gc.collect()

In [ ]:
#del train["hour_sine"]
#del train["hour_cosine"]
del train["device_hour_cnt"]
#gc.collect()
del train["os_hour_cnt"]
del train["app_hour_cnt"]
del train["ip_hour_cnt"]
del train["channel_hour_cnt"]
gc.collect()

In [ ]:
for col in ['device_cnt', 'ip_cnt', 'os_cnt', 'ip_cnt', 'app_cnt', 'channel_cnt']:
    train[col] = train[col].astype(np.uint32)

In [ ]:
features = ['ip', 'app', 'device', 'os', 'channel', 'device_cnt', 'ip_cnt', 'os_cnt', 'ip_cnt', 'app_cnt', 'channel_cnt', 
            'hourofday', 'minuteofhour', 'hour_cnt','device_hour_cntphour','ip_hour_cntphour', 'channel_hour_cntphour',
            'os_hour_cntphour','app_hour_cntphour' ]
shuffle_crossvalidator(model, train[features], train.is_attributed, cvlist=cvlist1)

In [19]:
gc.collect()

0

In [ ]:
del train["device_hour_cntphour"]
#gc.collect()
del train["os_hour_cntphour"]
del train["app_hour_cntphour"]
del train["ip_hour_cntphour"]
del train["channel_hour_cntphour"]
gc.collect()

In [ ]:
cnt_enc = TargetEncoder(cols=['device'], targetcol='is_attributed', func='mean', cname='device_mean', add_to_orig=True)
train = cnt_enc.fit_transform(train)
cnt_enc = TargetEncoder(cols=['os'], targetcol='is_attributed', func='mean', cname='os_mean', add_to_orig=True)
train = cnt_enc.fit_transform(train)
cnt_enc = TargetEncoder(cols=['app'], targetcol='is_attributed', func='mean', cname='app_mean', add_to_orig=True)
train = cnt_enc.fit_transform(train)
cnt_enc = TargetEncoder(cols=['ip'], targetcol='is_attributed', func='mean', cname='ip_mean', add_to_orig=True)
train = cnt_enc.fit_transform(train)
cnt_enc = TargetEncoder(cols=['channel'], targetcol='is_attributed', func='mean', cname='channel_mean', add_to_orig=True)
train = cnt_enc.fit_transform(train)

In [ ]:
features = ['ip', 'app', 'device', 'os', 'channel', 'device_cnt', 'ip_cnt', 'os_cnt', 'ip_cnt', 'app_cnt', 'channel_cnt', 
            'hourofday', 'minuteofhour', 'hour_cnt','device_mean','os_mean', 'app_mean',
            'ip_mean','channel_mean' ]
shuffle_crossvalidator(model, train[features], train.is_attributed, cvlist=cvlist1)

In [ ]:
cnt_enc = TargetEncoder(cols=['device', 'hourofday'], targetcol='is_attributed', func='mean', cname='device_hour_mean', add_to_orig=True)
train = cnt_enc.fit_transform(train)
cnt_enc = TargetEncoder(cols=['os', 'hourofday'], targetcol='is_attributed', func='mean', cname='os_hour_mean', add_to_orig=True)
train = cnt_enc.fit_transform(train)
cnt_enc = TargetEncoder(cols=['app', 'hourofday'], targetcol='is_attributed', func='mean', cname='app_hour_mean', add_to_orig=True)
train = cnt_enc.fit_transform(train)
cnt_enc = TargetEncoder(cols=['ip', 'hourofday'], targetcol='is_attributed', func='mean', cname='ip_hour_mean', add_to_orig=True)
train = cnt_enc.fit_transform(train)
cnt_enc = TargetEncoder(cols=['channel', 'hourofday'], targetcol='is_attributed', func='mean', cname='channel_hour_mean', add_to_orig=True)
train = cnt_enc.fit_transform(train)

In [ ]:
%%time
features = ['ip', 'app', 'device', 'os', 'channel', 'device_cnt', 'ip_cnt', 'os_cnt', 'ip_cnt', 'app_cnt', 'channel_cnt', 
            'hourofday', 'minuteofhour', 'hour_cnt','device_mean','os_mean', 'app_mean',
            'ip_mean','channel_mean', 'device_hour_mean','os_hour_mean', 'app_hour_mean',
            'ip_hour_mean','channel_hour_mean' ]
shuffle_crossvalidator(model, train[features], train.is_attributed, cvlist=cvlist1)